In [36]:
library("fitzRoy")
library(magrittr)
library(dplyr)

In [37]:
get_afl_detailed_match_data <- function(matchid, roundid, competitionid,
                                         cookie) {
  url <- "http://api.afl.com.au/cfs/afl/statsCentre/teams"
  request_metadata <- httr::GET(url,
    query = list(
      matchId = matchid,
      roundId = roundid,
      competitionId = competitionid
    ),
    httr::add_headers(`X-media-mis-token` = cookie)
  ) %>%
    httr::content(as = "text", encoding = "UTF-8")

  # Check that round info is available on web, if not return error
  if (stringr::str_detect(request_metadata, "Page Not Found")) {
    stop(paste0(
      "Invalid match ID (", matchid, "). Have you checked that this ",
      "game has been played yet?"
    ))
  }
  match_data <- request_metadata %>%
    jsonlite::fromJSON(flatten = TRUE) %>%
    .$lists %>%
    dplyr::as_tibble() %>%
    dplyr::mutate(
      Match.Id = matchid,
      Round.Id = roundid,
      Competition.Id = competitionid
    ) %>%
    # Assumption: row 1 is home, row 2 is away. Have tested for correctness.
    dplyr::mutate(home.away = c("Home", "Away")) %>%
    tidyr::gather(
      "stat", "value",
      .data$stats.averages.goals:.data$team.teamNickname
    ) %>%
    dplyr::mutate(stat = dplyr::case_when(
      .data$home.away == "Home" ~ stringr::str_c("home.", .data$stat),
      .data$home.away == "Away" ~ stringr::str_c("away.", .data$stat)
    )) %>%
    dplyr::select(-.data$home.away) %>%
    tidyr::spread(.data$stat, .data$value) %>%
    dplyr::rename_at(
      dplyr::vars(dplyr::contains(".lastUpdated")),
      dplyr::funs(stringr::str_replace(., "stats", "STATS"))
    ) %>%
    dplyr::mutate_at(
      dplyr::vars(tidyselect::contains(".stats", ignore.case = FALSE)),
      readr::parse_number
    ) %>%
    dplyr::rename_at(
      dplyr::vars(tidyselect::contains(".lastUpdated")),
      dplyr::funs(stringr::str_replace(., "STATS", "stats"))
    ) %>%
    dplyr::mutate_at(
      dplyr::vars(tidyselect::contains(".lastUpdated")),
      readr::parse_datetime
    )
}

In [38]:
get_afl_detailed_data <- function(matchids) {
  cookie <- get_afl_cookie()
  # Round and competition IDs can be inferred from match Ids:
  # Match ID:       "CD_M20172640101"
  # Round ID:       "CD_R201726401"     M->R, last two characters removed
  # Competition ID: "CD_S2017264"       R->S, last two characters removed
  roundid_vector <- matchids %>%
    stringr::str_sub(1, -3) %>% # Remove last two characters
    stringr::str_replace("M", "R") # Replace R with S
  compid_vector <- roundid_vector %>%
    stringr::str_sub(1, -3) %>% # Remove last two characters
    stringr::str_replace("R", "S")
  purrr::pmap_dfr(
    list(matchids, roundid_vector, compid_vector),
    ~ get_afl_detailed_match_data(..1, ..2, ..3, cookie)
  )
}

In [39]:
get_afl_match_data <- function(start_year, end_year) {
  .Deprecated("fetch_results_afl")
    if (end_year == 2022) {
          end_year <- Sys.Date() %>%
    format("%Y") %>%
    as.numeric()
    }
    seasons <- start_year:end_year

    

  seasons %>%
    purrr::map_dfr(fetch_results_afl, NULL, "AFLM")
}

In [ ]:
fat_data_2022<-get_afl_match_data(2022, 2022)

In [40]:
slim_season_table<-get_afl_match_data(2022, 2022)

Warning message:
“'get_afl_match_data' is deprecated.
Use 'fetch_results_afl' instead.
See help("Deprecated")”


In [41]:
slim_season_table

match.name,match.date,match.status,match.matchId,match.venue,match.utcStartTime,match.homeTeamId,match.awayTeamId,match.round,match.venueLocalStartTime,⋯,homeTeamScoreChart.rightPosters,homeTeamScoreChart.rushedBehinds,homeTeamScoreChart.touchedBehinds,awayTeamScoreChart.goals,awayTeamScoreChart.leftBehinds,awayTeamScoreChart.rightBehinds,awayTeamScoreChart.leftPosters,awayTeamScoreChart.rightPosters,awayTeamScoreChart.rushedBehinds,awayTeamScoreChart.touchedBehinds
<chr>,<dttm>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
Melbourne Vs Western Bulldogs,2022-03-16 08:10:00,CONCLUDED,CD_M20220140101,CD_V40,2022-03-16T08:10:00,CD_T90,CD_T140,CD_R202201401,2022-03-16T19:10:00,⋯,1,2,0,11,2,2,0,0,1,0
Carlton Vs Richmond,2022-03-17 08:25:00,CONCLUDED,CD_M20220140102,CD_V40,2022-03-17T08:25:00,CD_T30,CD_T120,CD_R202201401,2022-03-17T19:25:00,⋯,0,1,0,11,3,5,0,0,2,0
St Kilda Vs Collingwood,2022-03-18 08:50:00,CONCLUDED,CD_M20220140103,CD_V190,2022-03-18T08:50:00,CD_T130,CD_T40,CD_R202201401,2022-03-18T19:50:00,⋯,0,2,0,15,5,3,2,1,1,0
Geelong Cats Vs Essendon,2022-03-19 03:10:00,CONCLUDED,CD_M20220140104,CD_V40,2022-03-19T03:10:00,CD_T70,CD_T50,CD_R202201401,2022-03-19T14:10:00,⋯,1,6,0,11,3,1,0,0,2,0
GWS Giants Vs Sydney Swans,2022-03-19 06:10:00,CONCLUDED,CD_M20220140105,CD_V9,2022-03-19T06:10:00,CD_T1010,CD_T160,CD_R202201401,2022-03-19T17:10:00,⋯,1,3,0,17,2,4,0,0,4,0
Brisbane Lions Vs Port Adelaide,2022-03-19 09:10:00,CONCLUDED,CD_M20220140106,CD_V20,2022-03-19T09:10:00,CD_T20,CD_T110,CD_R202201401,2022-03-19T19:10:00,⋯,0,3,0,10,2,5,1,0,1,0
Hawthorn Vs North Melbourne,2022-03-20 02:10:00,CONCLUDED,CD_M20220140107,CD_V40,2022-03-20T02:10:00,CD_T80,CD_T100,CD_R202201401,2022-03-20T13:10:00,⋯,0,3,0,8,2,4,0,1,2,1
Adelaide Crows Vs Fremantle,2022-03-20 05:10:00,CONCLUDED,CD_M20220140108,CD_V6,2022-03-20T05:10:00,CD_T10,CD_T60,CD_R202201401,2022-03-20T15:40:00,⋯,1,0,0,11,8,6,1,0,2,0
West Coast Eagles Vs Gold Coast Suns,2022-03-20 08:40:00,CONCLUDED,CD_M20220140109,CD_V2925,2022-03-20T08:40:00,CD_T150,CD_T1000,CD_R202201401,2022-03-20T16:40:00,⋯,1,2,1,16,3,2,2,0,3,1


In [42]:
list_of_ids = slim_season_table[4]
#list_of_ids_2022 = fat_data_2022[4]

In [43]:
string_of_ids = ""
string_of_ids_2022 = ""
list_of_ids = unlist(list_of_ids)
ids = paste(list_of_ids, string_of_ids, sep="")

In [44]:
all_games_this_year = get_afl_detailed_data(ids)
all_games_this_year

Match.Id,Round.Id,Competition.Id,away.stats.averages.behinds,away.stats.averages.bounces,away.stats.averages.clangers,away.stats.averages.clearances.centreClearances,away.stats.averages.clearances.stoppageClearances,away.stats.averages.clearances.totalClearances,away.stats.averages.contestedMarks,⋯,home.stats.totals.superGoals,home.stats.totals.tackles,home.stats.totals.tacklesInside50,home.stats.totals.totalPossessions,home.stats.totals.turnovers,home.stats.totals.uncontestedPossessions,home.team.teamAbbr,home.team.teamId,home.team.teamName,home.team.teamNickname
<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>
CD_M20220140101,CD_R202201401,CD_S2022014,11.4,7.2,55.6,14.2,26.0,40.1,11.0,⋯,NA,46,7,350,66,204,MELB,CD_T90,Melbourne,Demons
CD_M20220140102,CD_R202201401,CD_S2022014,11.7,8.2,65.4,13.6,19.8,33.4,11.3,⋯,NA,42,14,360,60,225,CARL,CD_T30,Carlton,Blues
CD_M20220140103,CD_R202201401,CD_S2022014,10.5,6.4,62.8,11.0,23.4,34.4,10.7,⋯,NA,51,10,365,72,221,STK,CD_T130,St Kilda,Saints
CD_M20220140104,CD_R202201401,CD_S2022014,11.0,7.1,56.6,13.3,20.5,33.8,10.0,⋯,NA,37,2,385,55,234,GEEL,CD_T70,Geelong Cats,Cats
CD_M20220140105,CD_R202201401,CD_S2022014,12.1,5.0,59.8,11.3,25.9,37.2,11.2,⋯,NA,62,5,334,56,191,GWS,CD_T1010,GWS Giants,Giants
CD_M20220140106,CD_R202201401,CD_S2022014,10.9,4.4,62.2,11.7,26.4,38.1,10.6,⋯,NA,57,10,335,73,194,BL,CD_T20,Brisbane Lions,Lions
CD_M20220140107,CD_R202201401,CD_S2022014,8.1,4.3,61.6,12.3,24.0,36.4,10.1,⋯,NA,62,14,363,71,231,HAW,CD_T80,Hawthorn,Hawks
CD_M20220140108,CD_R202201401,CD_S2022014,10.6,4.2,57.0,11.1,27.0,38.1,10.2,⋯,NA,51,9,362,61,219,ADEL,CD_T10,Adelaide Crows,Crows
CD_M20220140109,CD_R202201401,CD_S2022014,11.1,6.2,63.0,12.7,26.5,39.2,9.6,⋯,NA,38,5,333,77,211,WCE,CD_T150,West Coast Eagles,Eagles


In [45]:
match('home.stats.totals.behinds',cols)

[1] 146

In [46]:
cols = colnames(all_games_this_year)
wanted_away_stats = cols[50:99]
wanted_home_stats = cols[146:195]
wanted_stats = c(wanted_away_stats, wanted_home_stats)
wanted_stats
cols

[1] "away.stats.totals.behinds"                                
  [2] "away.stats.totals.bounces"                                
  [3] "away.stats.totals.clangers"                               
  [4] "away.stats.totals.clearances.centreClearances"            
  [5] "away.stats.totals.clearances.stoppageClearances"          
  [6] "away.stats.totals.clearances.totalClearances"             
  [7] "away.stats.totals.contestedMarks"                         
  [8] "away.stats.totals.contestedPossessions"                   
  [9] "away.stats.totals.disposalEfficiency"                     
 [10] "away.stats.totals.disposals"                              
 [11] "away.stats.totals.dreamTeamPoints"                        
 [12] "away.stats.totals.extendedStats"                          
 [13] "away.stats.totals.freesAgainst"                           
 [14] "away.stats.totals.freesFor"                               
 [15] "away.stats.totals.goalAccuracy"                           
 [16] "away.stats.totals.goalAssists"                            
 [17] "away.stats.totals.goalEfficiency"                         
 [18] "away.stats.totals.goals"                                  
 [19] "away.stats.totals.handballs"                              
 [20] "away.stats.totals.hitouts"                                
 [21] "away.stats.totals.inside50s"                              
 [22] "away.stats.totals.intercepts"                             
 [23] "away.stats.totals.interchangeCounts.interchangeCap"       
 [24] "away.stats.totals.interchangeCounts.interchangeCountQ1"   
 [25] "away.stats.totals.interchangeCounts.interchangeCountQ2"   
 [26] "away.stats.totals.interchangeCounts.interchangeCountQ3"   
 [27] "away.stats.totals.interchangeCounts.interchangeCountQ4"   
 [28] "away.stats.totals.interchangeCounts.totalInterchangeCount"
 [29] "away.stats.totals.kicks"                                  
 [30] "away.stats.totals.lastUpdated"                            
 [31] "away.stats.totals.marks"                                  
 [32] "away.stats.totals.marksInside50"                          
 [33] "away.stats.totals.metresGained"                           
 [34] "away.stats.totals.onePercenters"                          
 [35] "away.stats.totals.ranking"                                
 [36] "away.stats.totals.ratingPoints"                           
 [37] "away.stats.totals.rebound50s"                             
 [38] "away.stats.totals.scoreInvolvements"                      
 [39] "away.stats.totals.shotEfficiency"                         
 [40] "away.stats.totals.shotsAtGoal"                            
 [41] "away.stats.totals.superGoals"                             
 [42] "away.stats.totals.tackles"                                
 [43] "away.stats.totals.tacklesInside50"                        
 [44] "away.stats.totals.totalPossessions"                       
 [45] "away.stats.totals.turnovers"                              
 [46] "away.stats.totals.uncontestedPossessions"                 
 [47] "away.team.teamAbbr"                                       
 [48] "away.team.teamId"                                         
 [49] "away.team.teamName"                                       
 [50] "away.team.teamNickname"                                   
 [51] "home.stats.totals.behinds"                                
 [52] "home.stats.totals.bounces"                                
 [53] "home.stats.totals.clangers"                               
 [54] "home.stats.totals.clearances.centreClearances"            
 [55] "home.stats.totals.clearances.stoppageClearances"          
 [56] "home.stats.totals.clearances.totalClearances"             
 [57] "home.stats.totals.contestedMarks"                         
 [58] "home.stats.totals.contestedPossessions"                   
 [59] "home.stats.totals.disposalEfficiency"                     
 [60] "home.stats.totals.disposals"                              
 [61] "home.stats.totals.dreamTeamPoints" 

[1] "Match.Id"                                                   
  [2] "Round.Id"                                                   
  [3] "Competition.Id"                                             
  [4] "away.stats.averages.behinds"                                
  [5] "away.stats.averages.bounces"                                
  [6] "away.stats.averages.clangers"                               
  [7] "away.stats.averages.clearances.centreClearances"            
  [8] "away.stats.averages.clearances.stoppageClearances"          
  [9] "away.stats.averages.clearances.totalClearances"             
 [10] "away.stats.averages.contestedMarks"                         
 [11] "away.stats.averages.contestedPossessions"                   
 [12] "away.stats.averages.disposalEfficiency"                     
 [13] "away.stats.averages.disposals"                              
 [14] "away.stats.averages.dreamTeamPoints"                        
 [15] "away.stats.averages.extendedStats"                          
 [16] "away.stats.averages.freesAgainst"                           
 [17] "away.stats.averages.freesFor"                               
 [18] "away.stats.averages.goalAccuracy"                           
 [19] "away.stats.averages.goalAssists"                            
 [20] "away.stats.averages.goalEfficiency"                         
 [21] "away.stats.averages.goals"                                  
 [22] "away.stats.averages.handballs"                              
 [23] "away.stats.averages.hitouts"                                
 [24] "away.stats.averages.inside50s"                              
 [25] "away.stats.averages.intercepts"                             
 [26] "away.stats.averages.interchangeCounts.interchangeCap"       
 [27] "away.stats.averages.interchangeCounts.interchangeCountQ1"   
 [28] "away.stats.averages.interchangeCounts.interchangeCountQ2"   
 [29] "away.stats.averages.interchangeCounts.interchangeCountQ3"   
 [30] "away.stats.averages.interchangeCounts.interchangeCountQ4"   
 [31] "away.stats.averages.interchangeCounts.totalInterchangeCount"
 [32] "away.stats.averages.kicks"                                  
 [33] "away.stats.averages.lastUpdated"                            
 [34] "away.stats.averages.marks"                                  
 [35] "away.stats.averages.marksInside50"                          
 [36] "away.stats.averages.metresGained"                           
 [37] "away.stats.averages.onePercenters"                          
 [38] "away.stats.averages.ranking"                                
 [39] "away.stats.averages.ratingPoints"                           
 [40] "away.stats.averages.rebound50s"                             
 [41] "away.stats.averages.scoreInvolvements"                      
 [42] "away.stats.averages.shotEfficiency"                         
 [43] "away.stats.averages.shotsAtGoal"                            
 [44] "away.stats.averages.superGoals"                             
 [45] "away.stats.averages.tackles"                                
 [46] "away.stats.averages.tacklesInside50"                        
 [47] "away.stats.averages.totalPossessions"                       
 [48] "away.stats.averages.turnovers"                              
 [49] "away.stats.averages.uncontestedPossessions"                 
 [50] "away.stats.totals.behinds"                                  
 [51] "away.stats.totals.bounces"                                  
 [52] "away.stats.totals.clangers"                                 
 [53] "away.stats.totals.clearances.centreClearances"              
 [54] "away.stats.totals.clearances.stoppageClearances"            
 [55] "away.stats.totals.clearances.totalClearances"               
 [56] "away.stats.totals.contestedMarks"                           
 [57] "away.stats.totals.contestedPossessions"                     
 [58] "away.stats.totals.disposalEfficiency"                       
 [59] "away.stats.totals.disposals"                       

In [ ]:
detailed_table = select(all_games_this_year, wanted_stats)

In [47]:
table = select(all_games_this_year, wanted_stats)

In [48]:
write.csv(table,"/Users/somduttasinha/Google Drive/Personal Projects/AFL-Stats/team_stats/match_stats_2022.csv", row.names = FALSE)

In [ ]:
fetch_player_details_afl(2022, team = NULL, comp = "AFLM")

In [ ]:
fetch_player_details("Hawthorn")

In [ ]:
fetch_lineup_afl(season = 2021, round_number = NULL, comp = "AFLM")

In [ ]:
cookie <- get_afl_cookie()

In [ ]:
cookie

In [ ]:
avg_table

In [17]:
fetch_player_details_afl(season=2013, team = NULL, comp = "AFLM")

ℹ Fetching player details for Adelaide Crows, 2013
✔ Fetching player details for Adelaide Crows, 2013 ... done


ℹ Fetching player details for Brisbane Lions, 2013
✔ Fetching player details for Brisbane Lions, 2013 ... done


ℹ Fetching player details for Collingwood, 2013
✔ Fetching player details for Collingwood, 2013 ... done


ℹ Fetching player details for Gold Coast Suns, 2013
✔ Fetching player details for Gold Coast Suns, 2013 ... done


ℹ Fetching player details for Carlton, 2013
✔ Fetching player details for Carlton, 2013 ... done


ℹ Fetching player details for North Melbourne, 2013
✔ Fetching player details for North Melbourne, 2013 ... done


ℹ Fetching player details for Port Adelaide, 2013
✔ Fetching player details for Port Adelaide, 2013 ... done


ℹ Fetching player details for Western Bulldogs, 2013
✔ Fetching player details for Western Bulldogs, 2013 ... done


ℹ Fetching player details for Hawthorn, 2013
✔ Fetching player details for Hawthorn, 2013 ... done


ℹ Fetchin

firstName,surname,id,team,season,jumperNumber,position,providerId,dateOfBirth,draftYear,heightInCm,weightInKg,recruitedFrom,debutYear,draftType,draftPosition,data_accessed
<chr>,<chr>,<int>,<chr>,<dbl>,<int>,<chr>,<chr>,<chr>,<chr>,<int>,<int>,<chr>,<chr>,<chr>,<chr>,<date>
Scott,Thompson,2,Adelaide Crows,2013,5,MIDFIELDER,CD_I210016,1983-03-14,2000,185,86,North Haven (SA)/Port Adelaide (SANFL)/Melbourne,2001,nationalDraft,16,2022-08-02
Graham,Johncock,1132,Adelaide Crows,2013,18,MEDIUM_FORWARD,CD_I210056,1982-10-21,2000,178,82,Port Adelaide (SANFL),2002,nationalDraft,67,2022-08-02
Brent,Reilly,31,Adelaide Crows,2013,3,MEDIUM_DEFENDER,CD_I220086,1983-11-12,2001,183,84,Gisborne (Vic)/Calder U18,2002,nationalDraft,12,2022-08-02
Ben,Rutten,43,Adelaide Crows,2013,25,KEY_DEFENDER,CD_I220093,1983-05-28,2002,190,98,West Adelaide (SA),2003,rookieElevation,NA,2022-08-02
Jason,Porplyzia,3,Adelaide Crows,2013,40,MEDIUM_FORWARD,CD_I230174,1984-11-27,2005,179,80,West Adelaide (SA),2006,preseason,9,2022-08-02
Richard,Tambling,1039,Adelaide Crows,2013,1,MEDIUM_DEFENDER,CD_I230216,1986-09-12,2004,180,89,Southern Districts (NT)/Richmond,2005,nationalDraft,4,2022-08-02
Richard,Douglas,39,Adelaide Crows,2013,26,MIDFIELDER,CD_I240370,1987-02-06,2005,181,79,Broadford (Vic)/Calder U18,2006,nationalDraft,16,2022-08-02
Nathan,Van Berlo,4,Adelaide Crows,2013,7,MIDFIELDER,CD_I250118,1986-06-02,2004,185,80,Whitfords (WA)/West Perth (WAFL),2005,nationalDraft,24,2022-08-02
David,Mackay,38,Adelaide Crows,2013,14,MIDFIELDER,CD_I250362,1988-07-25,2006,181,77,Beverley Hills (Vic)/Trinity Grammar (Vic)/Oakleigh U18,2008,nationalDraft,48,2022-08-02
